In [2]:
import pandas as pd
import csv
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [33]:
df = pd.read_csv('/content/drive/My Drive/datasets/titanic.zip')

In [34]:
df.head()

,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,zero.5,zero.6,Parch,zero.7,zero.8,zero.9,zero.10,zero.11,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,2.0,0,0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0,0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,2.0,0,0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2.0,0,0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,2.0,0,0,0


In [36]:
lis = [f'zero.{n}' for n in range(1,19)]
lis.append('zero')

In [37]:
df.drop(lis, axis=1, inplace=True)
df.dropna(axis=0, inplace=True)

In [38]:
df.head(10)

,Passengerid,Age,Fare,Sex,sibsp,Parch,Pclass,Embarked,2urvived
0,1,22.0,7.2500,0,1,0,3,2.0,0
1,2,38.0,71.2833,1,1,0,1,0.0,1
2,3,26.0,7.9250,1,0,0,3,2.0,1
3,4,35.0,53.1000,1,1,0,1,2.0,1
4,5,35.0,8.0500,0,0,0,3,2.0,0
5,6,28.0,8.4583,0,0,0,3,1.0,0
6,7,54.0,51.8625,0,0,0,1,2.0,0
7,8,2.0,21.0750,0,3,1,3,2.0,0
8,9,27.0,11.1333,1,0,2,3,2.0,1
9,10,14.0,30.0708,1,1,0,2,0.0,1


In [39]:
df.columns

Index(['Passengerid', 'Age', 'Fare', 'Sex', 'sibsp', 'Parch', 'Pclass',
       'Embarked', '2urvived'],
      dtype='object')

In [ ]:
df.isnull().sum()

In [42]:
X = df[['Age', 'Fare', 'Sex', 'sibsp', 'Parch', 'Pclass',
       'Embarked']]
y = df.pop('2urvived')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [43]:
num_c = ['Age','Fare', 'Parch','sibsp']
bucket_c = ['Age'] # mix col with Sex

cat_i_c = ['Sex','Pclass', 'Embarked']
# cat_e_c = [‘Ticket’]

In [21]:
def make_input(df_data, df_label, epoch=10, shuffle=True, batch_size=100):
  def input_fn():
    ds = tf.data.Dataset.from_tensor_slices((dict(df_data), df_label))
    if shuffle:
      ds.shuffle(1000)
    return ds.batch(batch_size).repeat(epoch)
  return input_fn

In [44]:
def scale(feature):
  def minmax(x):
    mini = X_train[feature].min()
    maxx = X_train[feature].max()
    return (x - mini)/(maxx-mini)
  return minmax

In [65]:
X_train['Embarked'] = X_train['Embarked'].astype('int64')
X_test['Embarked'] = X_test['Embarked'].astype('int64')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
X_train.head()

In [57]:
feat_col = []

# numeric columns
for feat in num_c:
  scal_input = scale(feat)
  feat_col.append(tf.feature_column.numeric_column(feat, normalizer_fn=scal_input))

# bucketized columns
age = tf.feature_column.numeric_column('Age', dtype=tf.float32)
age_bucket = tf.feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feat_col.append(age_bucket)

# cat indicator columns
for feat in cat_i_c:
  vocab = X_train[feat].unique()
  cat_c = tf.feature_column.categorical_column_with_vocabulary_list(feat, vocab)
  one_hot = tf.feature_column.indicator_column(cat_c)
  feat_col.append(one_hot)

# embedded columns


# crossed columns
vocab = X_train['Sex'].unique()
sex = tf.feature_column.categorical_column_with_vocabulary_list('Sex', vocab)

crossed_feature = tf.feature_column.crossed_column([age_bucket, sex], hash_bucket_size=1000)
crossed_feature = tf.feature_column.indicator_column(crossed_feature)
feat_col.append(crossed_feature)
len(feat_col)



9

In [58]:
feat_col

[NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function scale.<locals>.minmax at 0x7f1bc1b8dea0>),
 NumericColumn(key='Fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function scale.<locals>.minmax at 0x7f1bc1c35950>),
 NumericColumn(key='Parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function scale.<locals>.minmax at 0x7f1bc1c35378>),
 NumericColumn(key='sibsp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=<function scale.<locals>.minmax at 0x7f1bc1c35488>),
 BucketizedColumn(source_column=NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(18, 25, 30, 35, 40, 45, 50, 55, 60, 65)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Sex', vocabulary_list=(0, 1), dtype=tf.int64, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Pclass', v

In [75]:
def make_input(df_data, df_label, epochs= 10, shuffle=True, batch_size= 32):
  def input_fn():
    ds = tf.data.Dataset.from_tensor_slices((dict(df_data),df_label))
    if shuffle:
      ds.shuffle(1000)
    return ds.batch(batch_size)
  return input_fn

In [76]:
train_input = make_input(X_train, y_train)
eval_input = make_input(X_test, y_test)

In [67]:
classifier = tf.estimator.LinearClassifier(feat_col, n_classes=2)
classifier.train(train_input)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp75juqnnj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. 

In [68]:
result = classifier.evaluate(eval_input)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-06-20T00:40:54Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp75juqnnj/model.ckpt-280
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.86000s
INFO:tensorflow:Finished evaluation at 2020-06-20-00:40:55
INFO:tensorflow:Saving dict for global step 280: accuracy = 0.7847222, accuracy_

In [70]:
print(pd.Series(result))

accuracy                  0.784722
accuracy_baseline         0.712963
auc                       0.815563
auc_precision_recall      0.643618
average_loss              0.471830
label/mean                0.287037
loss                      0.466536
precision                 0.771930
prediction/mean           0.236100
recall                    0.354839
global_step             280.000000
dtype: float64


In [71]:
# Using boosted model, this model does not take in embedding features

In [77]:
n_batches = 1
est = tf.estimator.BoostedTreesClassifier(feat_col, n_batches_per_layer=n_batches)

est.train(train_input)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpzexpzt9z', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


In [78]:
result = est.evaluate(eval_input)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-06-20T00:48:55Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpzexpzt9z/model.ckpt-27
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.08023s
INFO:tensorflow:Finished evaluation at 2020-06-20-00:48:56
INFO:tensorflow:Saving dict for global step 27: accuracy = 0.7476852, accuracy_baseline = 0.712963, auc = 0.7703315, auc_precision_recall = 0.5636673, average_loss = 0.50619435, global_step = 27, label/mean = 0.28703704, loss = 0.5024256, precision = 0.6415094, prediction/mean = 0.26799014, recall = 0.27419356
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 27: /tmp/tmpz

In [79]:
print(pd.Series(result))

accuracy                 0.747685
accuracy_baseline        0.712963
auc                      0.770332
auc_precision_recall     0.563667
average_loss             0.506194
label/mean               0.287037
loss                     0.502426
precision                0.641509
prediction/mean          0.267990
recall                   0.274194
global_step             27.000000
dtype: float64
